In [1]:
%pip install duckdb

import duckdb
import os
import glob
import pandas as pd
import numpy as np 



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.4 MB 6.1 MB/s 


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


# Relational model
For this project we have chosen the DuckDB database

In [3]:
#con = duckdb.connect(database='/content/drive/MyDrive/projecteADSDB/formatted/formatted.duckdb', read_only=False)
con = duckdb.connect(database='./formatted/formatted.duckdb', read_only=False)

## Table Creation
we create one table for each data source and version (timestamp)

In [4]:
#dir = '/content/drive/MyDrive/projecteADSDB/landing/persistent/'
dir = 'landing/persistent/'

#os.chdir(dir)
#os.chdir(dir)

datasources = [datasource for datasource in os.listdir(dir)]

# dictionary that contains all the table dataframes
dictdf = {}

for ds in datasources:
  files = [f for f in os.listdir(dir+'/'+ds)]
  for f in files:
    # omit '.csv'
    fname = f[:-4]
    con.execute(f'DROP TABLE IF EXISTS {fname}')
    con.execute(f"CREATE TABLE IF NOT EXISTS {fname} AS SELECT * FROM read_csv_auto('{dir}{ds}/{f}');")
    dictdf[fname] = con.execute(f"SELECT * FROM {fname}").df()


## Show table contents

In [5]:
for df in dictdf:
  print('\n---------------------------------')
  print(df)
  print('\n---------------------------------')
  print(dictdf[df].head(5))
  print(dictdf[df].describe())



---------------------------------
Governance_Data$1666445932

---------------------------------
  Country Name Country Code  \
0  Afghanistan          AFG   
1  Afghanistan          AFG   
2  Afghanistan          AFG   
3  Afghanistan          AFG   
4  Afghanistan          AFG   

                                         Series Name       Series Code  \
0                    Control of Corruption: Estimate            CC.EST   
1           Control of Corruption: Number of Sources         CC.NO.SRC   
2             Control of Corruption: Percentile Rank        CC.PER.RNK   
3  Control of Corruption: Percentile Rank, Lower ...  CC.PER.RNK.LOWER   
4  Control of Corruption: Percentile Rank, Upper ...  CC.PER.RNK.UPPER   

   1996 [YR1996]  2000 [YR2000]  2004 [YR2004]  2008 [YR2008]  2012 [YR2012]  \
0      -1.291705      -1.271724      -1.350647      -1.672096      -1.430373   
1       2.000000       2.000000       5.000000       7.000000      10.000000   
2       4.301075       4.787234

In [6]:
con.execute("SHOW TABLES").fetchall()

[('Country_Level_Data$1666445932',),
 ('Governance_Data$1666445932',),
 ('Olympics_Data$1666445932',)]

In [7]:
con.close()